## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from src.utils_rle import rle_decode


## 2. Importar dataset

In [ ]:
df = pd.read_csv("../data/train_split.csv")
df.columns = ["ImageId", "ClassId", "EncodedPixels"]

df.head()


## 3. Estadísticas generales

In [ ]:
print("Total de filas:", len(df))
print("Total de imágenes únicas:", df["ImageId"].nunique())
print("Clases:", df["ClassId"].unique())
df.describe()

## 4. Distribución de clases

plt.figure(figsize=(6,4))
sns.countplot(x=df["ClassId"])
plt.title("Distribución de defectos por clase")
plt.xlabel("Clase")
plt.ylabel("Cantidad")
plt.show()


## 5. Porcentaje por clase

In [ ]:
plt.figure(figsize=(6,4))
(df["ClassId"].value_counts(normalize=True) * 100).plot(kind="bar")
plt.title("Porcentaje de cada clase")
plt.ylabel("% del total")
plt.show()


## 6. Imágenes con vs sin defectos

In [ ]:
df["has_defect"] = df["EncodedPixels"].notnull()
sns.countplot(df["has_defect"])
plt.title("Imágenes con y sin defectos")
plt.show()

percentage = df["has_defect"].mean() * 100
print(f"Porcentaje de imágenes CON defectos: {percentage:.2f}%")


## 7. Número de defectos por imagen

In [ ]:
counts = df.groupby("ImageId")["ClassId"].nunique()

plt.figure(figsize=(6,4))
plt.hist(counts, bins=5)
plt.title("Número de defectos por imagen")
plt.xlabel("Cantidad de clases presentes")
plt.ylabel("Número de imágenes")
plt.show()


## 8. Visualización del dataset (imagen + máscara)

In [ ]:
def show_sample(image_id, img_dir="../data/train_images"):
    df_img = df[df["ImageId"] == image_id]

    img = cv2.imread(f"{img_dir}/{image_id}", cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))

    mask = np.zeros((256, 256, 4), dtype=np.uint8)

    for _, row in df_img.iterrows():
        if isinstance(row["EncodedPixels"], str):
            class_id = int(row["ClassId"]) - 1
            decoded = rle_decode(row["EncodedPixels"], shape=(256,1600))
            decoded = cv2.resize(decoded, (256,256))
            mask[:,:,class_id] = decoded

    overlay = img.copy()
    overlay = cv2.cvtColor(overlay, cv2.COLOR_GRAY2BGR)
    overlay[mask.sum(axis=-1) > 0] = [255,0,0]  # rojo

    fig, ax = plt.subplots(1,3, figsize=(14,4))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Imagen Original")
    ax[1].imshow(mask.sum(axis=-1))
    ax[1].set_title("Máscara (GT)")
    ax[2].imshow(overlay)
    ax[2].set_title("Overlay GT")
    plt.show()

# Mostrar 3 imágenes aleatorias
samples = df["ImageId"].sample(3).values
for s in samples:
    show_sample(s)
